In [ ]:
import pandas as pd
import os
# import scanpath as scp
import math
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import time
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
! cp -r /content/drive/MyDrive/scanpath_data /content/

In [ ]:
def get_file_data(file):
    file_path = os.path.join('/content/scanpath_data/scanpaths/', file)
    df = pd.read_csv(file_path, delimiter='\t')
    cols = df.columns.to_list()
    df['file'] = file.replace("_scanpath.tsv", "")
    df = df[['file'] + cols]
    return df
files = [i for i in os.listdir('/content/scanpath_data/scanpaths/')]
master_df = get_file_data(files[0])

for i  in  files[1:]:
    master_df = pd.concat([master_df, get_file_data(i)])

reader_data = pd.read_csv("/content/scanpath_data/reader_meta_mean_and_per-text.csv")

def get_scanpath(scan_record):
    scanpath = master_df[master_df['file'] == scan_record]
    scanpath = scanpath[['fixation_index', 'fixation_duration', 'next_saccade_duration', 'previous_saccade_duration', 'line', 'char_index_in_line',
                    'fixation_position_x', 'fixation_position_y', 'word_index_in_text', 'sent_index_in_text', 'char_index_in_text']]
    scanpath['change_in_word_flag'] = (scanpath['word_index_in_text'].diff() == 0).astype(int).shift(-1)
    scanpath['change_in_word'] = (scanpath['change_in_word_flag'] == 0).cumsum().shift(1).fillna(0)
    scanpath['same_word_next_saccade_duration'] = scanpath['change_in_word_flag']*scanpath['next_saccade_duration']
    scanpath['next_saccade_duration'] = scanpath['next_saccade_duration'] - scanpath['same_word_next_saccade_duration']
    scanpath['fixation_duration'] = scanpath['same_word_next_saccade_duration'] + scanpath['fixation_duration']
    scanpath = scanpath.drop('same_word_next_saccade_duration', axis =1)
    scanpath = scanpath.groupby(['change_in_word', 'word_index_in_text']).aggregate({'fixation_duration':'sum', 'next_saccade_duration':'sum', 'fixation_position_x':'mean', 'fixation_position_y':'mean'}).reset_index()
    scanpath['cum_fixation_duration'] = scanpath['fixation_duration'].cumsum().shift(1).fillna(0)
    scanpath = scanpath.reset_index()
    return scanpath

all_columns = ['file', 'fixation_index', 'text_domain', 'trial', 'acc_bq_1', 'acc_bq_2', 'acc_bq_3', 'acc_tq_1', 'acc_tq_2', 'acc_tq_3',
'fixation_duration', 'next_saccade_duration', 'previous_saccade_duration', 'version', 'line', 'roi', 'char_index_in_line',
'original_fixation_index', 'is_fixation_adjusted', 'reader_id', 'text_id', 'fixation_position_x', 'fixation_position_y',
'word_index_in_text', 'sent_index_in_text', 'char_index_in_text', 'word', 'character', 'text_id_numeric']

In [ ]:
# center_x = 840
# center_y = 525
# distance = 61
# unit = 0.0282

# files = master_df['file'].unique()
# files = [i for i in files if 'p0' in i]

# for i in range(len(files)-1):
#     scanpath1 = get_scanpath(files[i])
#     scanpath2 = get_scanpath(files[i+1])
#     print(files[i], files[i+1])
#     score, path, alignment, path_df  = rscasim(scanpath1, scanpath2, center_x, center_y, distance, unit, modulator=0.83)
#     # alignment = alignment.drop(0, axis=0).reset_index(drop=True)
#     alignment = alignment.merge(scanpath1[['index', 'fixation_position_x', 'fixation_position_y', 'fixation_duration', 'cum_fixation_duration']], left_on='s', right_on='index', how='left').rename(columns={'fixation_position_x':'fixation_position_x_s', 'fixation_position_y':'fixation_position_y_s',                                                                                                                                                                                         'fixation_duration':'fixation_duration_s', 'cum_fixation_duration':'cum_fixation_duration_s'}).drop('index', axis=1)
#     alignment = alignment.merge(scanpath2[['index', 'fixation_position_x', 'fixation_position_y', 'fixation_duration', 'cum_fixation_duration']], left_on='t', right_on='index', how='left').rename(columns={'fixation_position_x':'fixation_position_x_t', 'fixation_position_y':'fixation_position_y_t',
#                                                                                                                                                                                                             'fixation_duration':'fixation_duration_t', 'cum_fixation_duration':'cum_fixation_duration_t'}).drop('index', axis=1)

#     fig, axs = plt.subplots(2, 5, figsize=(8, 12))
#     # scanpath1 = scanpath1.iloc[:100, :]
#     # scanpath2 = scanpath2.iloc[:100, :]
#     axs = plot_scanpaths(axs, [scanpath1, scanpath2])
#     axs = plot_alignments(axs, alignment)

#     for ax in axs:
#         for axx in ax:
#             axx.tick_params(axis='x', rotation=90)

#     plt.tight_layout()
#     plt.pause(1)

In [ ]:
simmilarity_matrix_df = pd.DataFrame(data = simmilarity_matrix, columns=files, index=files)
simmilarity_matrix_df = simmilarity_matrix_df.iloc[:43, :43]
files_cluster = [files[i] for i in range(file_count)][:43]
simmilarity_matrix_np = simmilarity_matrix_df.to_numpy()
from sklearn.cluster import KMeans
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
kmeans.fit_predict(simmilarity_matrix_np)
cluster_labels = kmeans.labels_.tolist()
print("Cluster Labels:", cluster_labels)
cluster_df = pd.DataFrame(data=list(zip(files_cluster, cluster_labels)), columns=['file','cluster'])
cluster_df['readerID'] = cluster_df['file'].apply(lambda x: int(x.split("_")[0].replace("reader", "")))
reader_data_ = reader_data[['readerID', 'expert', 'meanPhyAccTQ', 'meanPhyAccBQ']]
df = cluster_df.merge(reader_data_, on="readerID").sort_values(by='cluster')
# df = df.groupby(["cluster", "expert"]).agg({"expert":"count"}).add_suffix("_count").reset_index()
# ll = cluster_df[cluster_df['cluster']==0]['file'].tolist()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering().fit(simmilarity_matrix_df)